# Pandas: Your Data's Best Friend
Two of the most useful things that Python can do for you as a reasercher is allow you to wrangle and visualize data quickly and responsively. We will be using the `pandas` module for data management. If you are interested, there are many tutorials out there which compliment what we are going to teach quite well, for example this Pandas tutorial on [Kaggle](https://www.kaggle.com/learn/pandas).

Let's start by importing these modules. We will use the shorthand `pd` for Pandas as a common abbreviation and  will make our lives just a little bit easier. We're also going to import numpy (as `np`) directly as we will need to access it later:

In [ ]:
import pandas as pd
import numpy as np

This workbook is organized to introduce various features, elements, techniques, and processes available to us using Pandas and Altair in research. We are going to walk through some of these steps using data that we have created, but mostly we are going to use and existing dataset to get used to working with real data that is often messy.

## VIA Rail Reliability
Our goal (case study, example) is to perform some analysis of the reliability of VIA Rail in 2019, particularly along a single corridor (the Kitchener corridor).

The data we will be working with consists of real and scheduled arrival and departure times of trains on portions of the VIA rail network in Canada. This data was collected from VIA's website, which reports these times as part of their customer service process. I have been collecting and storing this data for analysis, and we will use a year's worth of data for this example. 

For reference, here's a map of the railway system, complete with station names:
<img src="via_corridor.png">

## Pandas Data Frames
The meat and potatoes of data management in Pandas happens through the `DataFrame` object, and it's 1-dimensional cousin the `Series`. Data frames can be thought of just like tables in a database, or a single spreadsheet in an excel workbook. Series objects are like lists, and represent a single column in a data frame. There are many ways to create a data frame, here are common ones:
  * Read in data from a CSV file
  * Read in data from a Database
  * Create one in code
  
By far the most common approach is to read in a CSV file. Depending on the file size and the types of data, Pandas will automatically try to guess what types of data are in each column (strings, integers, floats), but you can tell it explicitly to make things run faster. We can view the size of the frame using the `shape` attribute, view the column names using the `columns` attribute, and the top rows of the data frame using the `head()` function.

### Creating Data Frames from Scratch
While not a common thing, sometimes it can be useful to create a data frame from scractch:

In [ ]:
votes = pd.DataFrame({'Motion': ['G-25', 'C-50', 'C-59'], 'For':[21, 15, 22], 'Against': [23, 29, 22]})
votes

In [ ]:
via = pd.read_csv('data/via_2019_all.csv')
display(via.shape)
display(via.columns)
via.head()

### Slicing and Selecting
You can view individual slices of the data by row or column. For example:

In [ ]:
display(via.iloc[250])
display(via.train.unique())
via[['train', 'prettyname']].head()

### Conditional Selection
A common thing you'll want to do is to filter the data you have based on a set of criteria. This can be done by passing a set of filtering arguments, treating the Data Frame a little like a dictionary.

For multiple criteria Pandas uses Python's *bitwise operators*, which are `&` for AND and `|` for OR. If your filtering criteria are complex, don't forget about DeMorgan's law!

In [ ]:
via[(via.train >= 80) & (via.train < 90)].head()

### Working with Subsets
You can assign subsets to new variables, but **be careful**. These subsets *sometimes* act as direct copies, sometimes they are simply selections on the larger frame. When you are making changes to the data, remember that if you want a true copy, you can use the `copy()` function.

In [ ]:
kitchener = via[(via.train >= 85) & (via.train < 90)].copy()

### Grouping Data
It's often useful to aggregate or group data together. We can do this using Pandas' `groupby` function. There are many different ways to do this, based on your application or how you use the output, but here's my approach:
  1. Slice your data frame so you have only the columns for grouping, and the columns for aggregating (at least one)
  2. Group on the groupby columns, use `as_index=False` to keep your dataframe flat.
  3. Provide an aggregation function (simple or complex) after your `groupby` call

In [ ]:
kitchener[['train', 'prettyname', 'schedArr']].groupby(['train', 'prettyname'], as_index=False).count().head(10)

### Sorting Values

In [ ]:
kitchener.sort_values(by='prettyname', ascending=True)

### Data Types
Much like with Python in general, we have to pay attention to data types in Pandas, and make sure that the data type that Pandas associates with a given column matches what we need it do be. Let's check the data types for our data set (note, the `object` type is what Pandas uses for string columns):

In [ ]:
kitchener.dtypes

What we'd *really* like is for the time columns to be treated as time, so we can do operations with them without too much fuss. Here's how we can convert:

In [ ]:
kitchener['schedArr'] = pd.to_datetime(kitchener.schedArr)
kitchener['realArr'] = pd.to_datetime(kitchener.realArr)
kitchener['schedDep'] = pd.to_datetime(kitchener.schedDep)
kitchener['realDep'] = pd.to_datetime(kitchener.realDep)
kitchener.dtypes

### Making New Columns
We can create new columns in the data based on other columns very easily:

In [ ]:
kitchener['arrDelta'] = kitchener['realArr'] - kitchener['schedArr']
kitchener['depDelta'] = kitchener['realDep'] - kitchener['schedDep']
display(kitchener.dtypes)
kitchener['arrDelta'] = kitchener['arrDelta'].dt.total_seconds()/60.0
kitchener['depDelta'] = kitchener['depDelta'].dt.total_seconds()/60.0
kitchener.head()

### Summaries
The `describe()` function can help us get some insights into our data very quickly, and let us know if there are any issues. We will use this summary function to determine some data integrity issues and to correct them.

In [ ]:
kitchener.describe()

## Example: Calculating On-Time Performance
As an example, let's calculate the on-time performance across all trains at all stations along the Kitchener corridor. We will define on-time performance as departing between 1 minute early and 5 minutes late (*Note: this is not how VIA defines their on-time performance*). We'll do this with the following steps:
  1. Create a subset of data with only on-time departures
  2. Group all departures by station name and train number
  3. Group on-time departures by station name and train number
  4. Merge the two datasets together
  5. Calculate the on-time performance

## Saving Files
Saving dataframes back into CSV files is quick and easy. We'll save our on-time performance data as well as our cleaned Kitchener data for later visualization use.